In [48]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import numpy as np

In [49]:
dtype = torch.FloatTensor
sentence = (
    'Lorem ipsum dolor sit amet consectetur adipisicing elit '
    'sed do eiusmod tempor incididunt ut labore et dolore magna '
    'aliqua Ut enim ad minim veniam quis nostrud exercitation'
)
word_num = {w:i for i,w in enumerate(list(set(sentence.split(' '))))}
num_word = {i:w for i,w in enumerate(list(set(sentence.split(' '))))}
n_class = len(word_num)
max_len = len(sentence.split())
n_hidden = 5

In [50]:
def make_batch(sentence):
    input_batch = []
    target_batch = []
    words = sentence.split()
    for i,word in enumerate(words[:-1]):
        input = [word_num[w] for w in words[:(i+1)]]
        input = input + [0] * (max_len-len(input))
        target = word_num[words[i+1]]
        input_batch.append(np.eye(n_class)[input])
        target_batch.append(target)
    return Variable(torch.Tensor(input_batch)),Variable(torch.Tensor(target_batch).type(torch.LongTensor))


In [51]:
class BiLSTM(nn.Module):
    #RNN layer包含两个参数 第一个是input的embedding_size 第二个是hidden层的输出size
    #input_size [seq_len,batch_size,embedding_size] -> [seq_len,batch_size,n_hidden]
    def __init__(self):
        super(BiLSTM,self).__init__()
        self.lstm = nn.LSTM(input_size=n_class,hidden_size=n_hidden,bidirectional=True)
        self.W = nn.Parameter(torch.randn(n_hidden*2,n_class).type(dtype))
        self.b = nn.Parameter(torch.randn(n_class).type(dtype))
        
    def forward(self,X):
        input = X.transpose(0,1)
        hidden_state = Variable(torch.zeros(1*2,len(X),n_hidden))# [num_layers(=1) * num_directions(=1), batch_size, n_hidden]
        cell_state = Variable(torch.zeros(1*2,len(X),n_hidden))# [num_layers(=1) * num_directions(=1), batch_size, n_hidden]
        outputs ,(_,_) = self.lstm(input,(hidden_state,cell_state))
        output = outputs[-1]#
        model = torch.mm(output,self.W)+self.b
        
        return model

In [55]:
input_batch,target_batch = make_batch(sentence)
model = BiLSTM()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr = 0.001)

for epoch in range(10000):
    optimizer.zero_grad()
    output = model(input_batch)
    loss = criterion(output,target_batch)
    if (epoch+1) % 100 == 0:
        print('epoch:%d , loss:%f' % (epoch+1,loss))
    loss.backward()
    optimizer.step()

epoch:100 , loss:3.799682
epoch:200 , loss:3.548778
epoch:300 , loss:3.359399
epoch:400 , loss:2.978504
epoch:500 , loss:2.664912
epoch:600 , loss:2.505166
epoch:700 , loss:2.362607
epoch:800 , loss:2.260327
epoch:900 , loss:2.155129
epoch:1000 , loss:2.060641
epoch:1100 , loss:1.976329
epoch:1200 , loss:1.897064
epoch:1300 , loss:1.821422
epoch:1400 , loss:1.746405
epoch:1500 , loss:1.677236
epoch:1600 , loss:1.606566
epoch:1700 , loss:1.528436
epoch:1800 , loss:1.449849
epoch:1900 , loss:1.380469
epoch:2000 , loss:1.317745
epoch:2100 , loss:1.258311
epoch:2200 , loss:1.205084
epoch:2300 , loss:1.157622
epoch:2400 , loss:1.114956
epoch:2500 , loss:1.079390
epoch:2600 , loss:1.044229
epoch:2700 , loss:1.011358
epoch:2800 , loss:0.980562
epoch:2900 , loss:0.951746
epoch:3000 , loss:0.924680
epoch:3100 , loss:0.899149
epoch:3200 , loss:0.874813
epoch:3300 , loss:0.847159
epoch:3400 , loss:0.815027
epoch:3500 , loss:0.770562
epoch:3600 , loss:1.934865
epoch:3700 , loss:1.350491
epoch:3800

In [ ]:
res = model(input_batch).data.max(1,keepdim=True)[1]
print(sentence)
print([num_word[n.item()] for n in res.squeeze()])